In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/HunEmPoli_fi/HunEmPoli_fi_train.tsv", sep = "\t")
df

,text,label,id
0,"Koska kaiken tämän sanoi mies, joka myönsi haa...",3,10796
1,He sanovat laskuttaneensa kuntaa murto-osasta ...,3,8987
2,"Tätä päättelyä hallituksen on pohdittava, pitä...",2,7605
3,HCSO on ottanut käyttöön erillisen luokan tämä...,2,8811
4,"Kolmanneksi teimme selväksi, että Ukrainan rak...",4,2337
...,...,...,...
15196,"On myönnettävä, että kuvaa 4 miljoonasta köyhy...",3,6702
15197,"Presidentti soitti kelloa), ja voitte nähdä, e...",4,5877
15198,Puolalaiset pitävät edelleen Transilvanian ruh...,4,14859
15199,"Olemme iloisia siitä, että nuori johtajamme ki...",4,9545


In [4]:
df = pd.read_csv("../data/gpt4_annotations/gpt4_annotations_test.tsv", sep = "\t")
df

,speech_id,sentence_id,text,label
0,http://ldf.fi/semparl/speeches/s2021_1_131_161,test_1,Ensimmäiseen käsittelyyn esitellään päiväjärje...,0
1,http://ldf.fi/semparl/speeches/s2021_1_131_161,test_2,Käsittelyn pohjana on maa- ja metsätalousvalio...,0
2,http://ldf.fi/semparl/speeches/s2021_1_131_161,test_3,Nyt päätetään lakiehdotusten sisällöstä.,0
3,http://ldf.fi/semparl/speeches/s2021_1_131_161,test_4,— Avaan yleiskeskustelun.,0
4,http://ldf.fi/semparl/speeches/s2021_1_131_161,test_5,"Edustaja Myllykoski, olkaa hyvä.",0
...,...,...,...,...
246,http://ldf.fi/semparl/speeches/s2019_1_027_095,test_247,Siellä on erilaista urheiluvälinelainausta ynn...,0
247,http://ldf.fi/semparl/speeches/s2019_1_027_095,test_248,"Voi sanoa, että kirjastot ovat kyllä oikeastaa...",2
248,http://ldf.fi/semparl/speeches/s2019_1_027_095,test_249,"Ja täytyy kyllä sanoa myöskin se, että kun min...",2
249,http://ldf.fi/semparl/speeches/s2019_1_027_095,test_250,"Elikkä on erinomaisen hienoa, että meillä on k...",1


In [5]:
# Check label distribution in original dataset
df["label"].value_counts()

label
0    155
2     43
1     15
5     12
6     12
3      6
8      5
7      2
4      1
Name: count, dtype: int64

In [6]:
hun_labels = {"neutral": 0, "fear": 1, "sadness": 2, "anger": 3, "disgust": 4, "success": 5, "joy": 6, "trust": 7}
finparl_labels = {"neutral": 0, "joy": 1, "trust": 2, "love": 3, "pos_surpirse": 4, "sadness": 5, "fear": 6, "anger": 7, "neg_surprise": 8}

# Replace Finparl lables with Many emotions labels
# First, drop labels 3,4 and 8 because they don't have a replacement
df = df[~df["label"].isin([3,4,8])]
df['label'] = df['label'].replace([6, 5, 7, 1, 2], [1, 2, 3, 4, 5])

# Check that label distribution mathces to make sure we made no errors.
df["label"].value_counts()

/tmp/tarkkaot/20519669/ipykernel_1229672/1761564333.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].replace([6, 5, 7, 1, 2], [1, 2, 3, 4, 5])


label
0    155
5     43
4     15
2     12
1     12
3      2
Name: count, dtype: int64

In [9]:
# We still have to make the column names match so Datasets worn't break
hun_columns = ["text", "label", "id"]

# To not lose any information, concatenate the two id columns into one id
#df["id"] = ["-".join([tup[0],str(tup[1])]) for tup in zip(df["speech_id"], df["sentence_id"])]
df["id"] = list(range(len(df)))

# Drop extra columns and reorder
df = df[hun_columns]
df

,text,label,id
0,Ensimmäiseen käsittelyyn esitellään päiväjärje...,0,0
1,Käsittelyn pohjana on maa- ja metsätalousvalio...,0,1
2,Nyt päätetään lakiehdotusten sisällöstä.,0,2
3,— Avaan yleiskeskustelun.,0,3
4,"Edustaja Myllykoski, olkaa hyvä.",0,4
...,...,...,...
246,Siellä on erilaista urheiluvälinelainausta ynn...,0,234
247,"Voi sanoa, että kirjastot ovat kyllä oikeastaa...",5,235
248,"Ja täytyy kyllä sanoa myöskin se, että kun min...",5,236
249,"Elikkä on erinomaisen hienoa, että meillä on k...",4,237


In [10]:
# All good! Save dataset!
df.to_csv("../data/HunEmPoli_fi/gpt4_annotations_hun_labels_test.tsv", sep = "\t", index = False)